### Intial Testing of the Dataset

In [114]:
import numpy as np
import pandas as pd 
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sentence_transformers import SentenceTransformer

In [115]:
df = pd.read_csv('data/imdb_top_1000.csv')
df.at[966,"Released_Year"] = 1995 #fixing the wrong value for apollo 13 
df.drop(columns = ["Poster_Link"], inplace= True)
df.dropna(inplace = True) #dropping null valued columns


In [116]:
#Converting to Numeric Values
df["Gross"] = df["Gross"].str.replace(",", "").astype("float")
df["Released_Year"] = df["Released_Year"].astype("int")
df["Runtime"] = df["Runtime"].str.replace(" min", "").astype("int")
df["IMDB_Rating"] = df["IMDB_Rating"].astype(float)
df["Meta_score"] = df["Meta_score"].astype(float)
df["No_of_Votes"] = df["No_of_Votes"].astype(float)
df.head()


,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,The Shawshank Redemption,1994,A,142,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110.0,28341469.0
1,The Godfather,1972,A,175,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367.0,134966411.0
2,The Dark Knight,2008,UA,152,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232.0,534858444.0
3,The Godfather: Part II,1974,A,202,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952.0,57300000.0
4,12 Angry Men,1957,U,96,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845.0,4360000.0


In [117]:
df["IMDB_Rating"] = smooth_target_encode(df, "IMDB_Rating", "Gross")
df["Meta_score"] = smooth_target_encode(df, "Meta_score", "Gross")
df["No_of_Votes"] = smooth_target_encode(df, "No_of_Votes", "Gross")
df["Certificate"] = smooth_target_encode(df, "Certificate", "Gross")
df["Genre"] = smooth_target_encode(df, "Genre", "Gross")
df["Director"] = smooth_target_encode(df, "Director", "Gross")
df["Star1"] = smooth_target_encode(df, "Star1", "Gross")
df["Star2"] = smooth_target_encode(df, "Star2", "Gross")
df["Star3"] = smooth_target_encode(df, "Star3", "Gross")
df["Star4"] = smooth_target_encode(df, "Star4", "Gross")
df["Series_Title"] = smooth_target_encode(df, "Series_Title", "Gross")




### Trying One Hot Encoding
Thoughts - Too many features \
Results - Not Tried

In [86]:
#One Hot encoding for Certificate, Genre, Director, Star1, Star2, Star3, Star 4
df_encoded = pd.get_dummies(df, columns = ["Certificate", "Genre", "Director", "Star1", "Star2", "Star3", "Star4"])
print(f"df_encoded_shape{df_encoded.shape}")
print(f"df_shape{df.shape}")

df_encoded_shape(714, 2953)
df_shape(714, 15)


### Trying Target Encoding

In [118]:
#defining functino to encode the target variable
def smooth_target_encode(df, col, target, smoothing_param =.3):
    mean_target = df[target].mean()
    encoded =df.groupby(col)[target].agg(["count", "mean"])
    counts = encoded["count"]
    means = encoded["mean"]
    smooth_encodings = (means*counts + mean_target*smoothing_param)/(counts+smoothing_param)
    return df[col].map(smooth_encodings)

In [119]:
df["IMDB_Rating"] = smooth_target_encode(df, "IMDB_Rating", "Gross")
df["Meta_score"] = smooth_target_encode(df, "Meta_score", "Gross")
df["No_of_Votes"] = smooth_target_encode(df, "No_of_Votes", "Gross")
df["Certificate"] = smooth_target_encode(df, "Certificate", "Gross")
df["Genre"] = smooth_target_encode(df, "Genre", "Gross")
df["Director"] = smooth_target_encode(df, "Director", "Gross")
df["Star1"] = smooth_target_encode(df, "Star1", "Gross")
df["Star2"] = smooth_target_encode(df, "Star2", "Gross")
df["Star3"] = smooth_target_encode(df, "Star3", "Gross")
df["Star4"] = smooth_target_encode(df, "Star4", "Gross")
df["Series_Title"] = smooth_target_encode(df, "Series_Title", "Gross")

In [ ]:
# Generating the Word embeddings for the Overview Column
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(df["Overview"].values, show_progress_bar=True)

No sentence-transformers model found with name sentence-transformers/all-MiniLM-L6-v2. Creating a new one with mean pooling.


Batches:   0%|          | 0/23 [00:00<?, ?it/s]

IndexError: invalid index to scalar variable.

In [ ]:
# Ensure all columns are numeric
for col in df_encoded.columns:
    if df_encoded[col].dtype == 'object':
        df_encoded[col] = pd.to_numeric(df_encoded[col], errors='coerce')




In [113]:
# Define features and target
X = df_encoded.drop(columns=["IMDB_Rating"]).values
y = df_encoded["IMDB_Rating"].values

# Standardize the features and target
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1)).flatten()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

NameError: name 'StandardScaler' is not defined

In [112]:
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
# Make predictions
y_pred = rf.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Display the first few predictions
print(f"Predictions: {y_pred[:5]}")

Mean Squared Error: 663725229908579.8
Predictions: [78494923.65374285 69294830.16114606 75534601.56356868 64045730.58592824
 69054663.75154759]
